In [12]:
import akshare as ak
import json
import pandas as pd
import sys
import datetime
import os
import numpy as np
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode
from pyecharts import options as opts
from pyecharts.charts import Scatter
from pyecharts.options import InitOpts
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line
import matplotlib.pyplot as plt
from pyecharts import options as opts
from pyecharts.charts import Bar
%matplotlib inline


def convert_time(x):
    return str(x).replace(' 00:00:00','')
    
import re
sys.path.append('..')
from configure.settings import DBSelector
db = DBSelector()
mongo = db.mongo('qq')
fund = mongo['fund']

In [2]:
def get_net_value(code):
    '''
    获取基金的净值
    '''
    fund_open_fund_info_em_df = ak.fund_open_fund_info_em(fund=code, indicator="单位净值走势")
    return fund_open_fund_info_em_df

In [3]:

def dump_mongo(df,code):
    js_str = df.to_json(orient='index')
    js_dict = json.loads(js_str)
    js_list = js_dict.values()
    try:
        fund[code].insert_many(js_list)
    except Exception as e:
        print(e)


In [6]:
def start_crawl(code):
    df = get_net_value(code)
    df['净值日期']=df['净值日期'].astype(str)
    dump_mongo(df,code)


In [7]:
for code in code_list:
    start_crawl(code)

In [ ]:
txt='''
1、$天弘中证银行指数A(F001594)$    ：跟踪中证银行指数，跟踪误差：0.10%。今年以来回报7.37%，年内最大回撤3.72%，最新规模25.67亿。

2、$易方达中证全指证券公司指数(LOF)(F502010)$    ：跟踪中证全指证券公司指数，跟踪误差：0.09%。今年以来回报-9.51%，年内最大回撤-9.85%，最新规模14.55亿。

3、$方正富邦中证保险主题指数(F167301)$    ：跟踪中证方正富邦保险主题指数，跟踪误差：0.14%。今年以来回报2.91%，年内最大回撤-5.53%，最新规模58.06亿。

4、$招商沪深300地产指数C(F013273)$    ：跟踪沪深300地产等权重指数，跟踪误差：0.20%。今年以来回报4.81%，年内最大回撤-6.16%，最新规模1.47亿。

5、$工银金融地产混合C(F010696)$    ：是一只偏股混合型主动基金，基金经理王君正和鄢耀。该基金今年以来回报1.15%，年内最大回撤-4.77%，最新规模7.09亿


'''

In [15]:
code_list = re.findall('\(F(\d+)\)',txt)
fund_name = re.findall('\$(.*?)\$',txt)
fund_dict = dict(zip(code_list,fund_name))
name_list  = list(fund_dict.values())
simple_name = [i.split('(')[0] for i in name_list]

In [70]:
fund_dict

{'001594': '天弘中证银行指数A(F001594)',
 '502010': '易方达中证全指证券公司指数(LOF)(F502010)',
 '167301': '方正富邦中证保险主题指数(F167301)',
 '013273': '招商沪深300地产指数C(F013273)',
 '010696': '工银金融地产混合C(F010696)'}

In [60]:
# after fetch data

In [24]:
def plot_profit_line(df,code):
    title="{}基金收益率曲线".format(code)
    X=df['净值日期'].tolist()
    Y=list(map(lambda x:round(x,2),df['单位净值'].tolist()))
    c = (
        Line()
        .add_xaxis(X)
        .add_yaxis('', Y, is_smooth=True,
            label_opts=opts.LabelOpts(is_show=False),
            linestyle_opts=opts.LineStyleOpts(width=1,color='rgb(255, 0, 0)'),
        ).set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(
                                    name='日期',
                                    # min_interval=5,
                                    splitline_opts=opts.SplitLineOpts(is_show=True),
                                            axislabel_opts=opts.LabelOpts(rotate=45),

                                    ),
            yaxis_opts=opts.AxisOpts(
                                    min_=min(Y),
                                    max_=max(Y),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            )
                                        ).set_colors(['green'])
        .render(f"../plot_image/{title}.html")
    )

In [67]:
W=10000
focus_num = [
3.98*W,
7519,
72,
309,
]

In [17]:
c = (
    Scatter(InitOpts(width='800px',height='500px'))
    .add_xaxis(simple_name)
    .add_yaxis('A',focus_num,label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
                                    # name='日期',
                                    # min_interval=5,
                                    splitline_opts=opts.SplitLineOpts(is_show=True),
                                            axislabel_opts=opts.LabelOpts(rotate=-45),

                                    ),
        yaxis_opts=opts.AxisOpts(    
            min_=0,
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
        title_opts=opts.TitleOpts(title="welcome"),
        visualmap_opts=opts.VisualMapOpts(type_="size",max_=150000, min_=2000),
    )
    .render("5fund.html")
)

In [69]:
grid = (
    Grid(init_opts=opts.InitOpts())
    .add(c, grid_opts=opts.GridOpts(pos_bottom="40%"))
    .render("grid_vertical5fund.html")
)

In [68]:

c = (
    Bar()
    .add_xaxis(
    new_name
    )
    .add_yaxis("基金关注度", focus_num,label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0)),
        title_opts=opts.TitleOpts(title="基金关注度"),
    )
    # .render("基金关注度.html")
)

In [65]:
new_name = []
for name in simple_name:
    new_name.append('\n'.join(list(name)))

In [73]:
new_name

['华\n安\n纳\n斯\n达\n克\n1\n0\n0\n指\n数',
 '博\n时\n标\n普\n5\n0\n0\nE\nT\nF\n联\n接',
 '天\n弘\n恒\n生\n科\n技\n指\n数\nA',
 '嘉\n实\n港\n股\n通\n新\n经\n济\n指\n数\nA',
 '华\n夏\n恒\n生\nE\nT\nF\n联\n接',
 '易\n方\n达\n中\n概\n互\n联\n5\n0\nE\nT\nF\n联\n接\n人\n民\n币\nA',
 '交\n银\n中\n证\n海\n外\n中\n国\n互\n联\n网\n指\n数',
 '华\n安\n香\n港\n精\n选',
 '中\n欧\n丰\n泓\n沪\n港\n深\nA']

In [16]:
def get_net_value_df(code):
    '''
    从mongodb获取数据
    '''
    result=[]
    for item in fund[code].find({},{'_id':0}):
        result.append(item)
    df = pd.DataFrame(result)
    return df

In [20]:
def get_year(start,end):
    '''
    获取年份
    '''
    year = (datetime.datetime.strptime(end,'%Y-%m-%d')- datetime.datetime.strptime(start,'%Y-%m-%d')).days/365
    return year

In [23]:
def get_profit_rate(df):
    '''
    计算所有的收益率
    '''
    return round((df['单位净值'].iloc[-1]-df['单位净值'].iloc[0])/df['单位净值'].iloc[0],2)

In [34]:
def get_yearly_profit_rate(df):
    '''
    计算年化收益率
    '''
    
    start = df['净值日期'].iloc[0]
    end = df['净值日期'].iloc[-1]
    year = get_year(start,end)
    print('开始时间：',start)
    print('成立年数：',round(year,2),'年')
    profit = get_profit_rate(df)
    print('成立以来累积收益率:',profit)
    year_profit = (1+profit)**(1/year)-1
    return start,round(year,2),profit,round(year_profit,2)



In [59]:

def fund_profit(code):
    '''
    生成字典
    '''

    df = get_net_value_df(code)
    max_withdraw,max_date_index = get_max_withdraw(df['单位净值'].tolist())
    start,year,profit,year_profit = get_yearly_profit_rate(df)
    d={}
    d['代码']=code
    d['名称']=fund_dict.get(code)
    d['发行日期']=start
    d['成立年数']=year
    d['累积收益率']=profit
    d['年化收益率']=year_profit
    d['最大回撤']=max_withdraw
    return d

In [60]:
result = []
for code in code_list:
    d = fund_profit(code)
    result.append(d)
    

开始时间： 2015-07-08
成立年数： 6.64 年
成立以来累积收益率: 0.31
开始时间： 2015-07-08
成立年数： 6.64 年
成立以来累积收益率: 0.21
开始时间： 2015-07-31
成立年数： 6.57 年
成立以来累积收益率: -0.21
开始时间： 2021-08-12
成立年数： 0.53 年
成立以来累积收益率: 0.08
开始时间： 2020-11-24
成立年数： 1.25 年
成立以来累积收益率: -0.01


In [61]:
profit_df = pd.DataFrame(result)

In [62]:
profit_df.head()

,代码,名称,发行日期,成立年数,累积收益率,年化收益率,最大回撤
0,001594,天弘中证银行指数A(F001594),2015-07-08,6.64,0.31,0.04,0.291909
1,502010,易方达中证全指证券公司指数(LOF)(F502010),2015-07-08,6.64,0.21,0.03,0.464186
2,167301,方正富邦中证保险主题指数(F167301),2015-07-31,6.57,-0.21,-0.04,0.497057
3,013273,招商沪深300地产指数C(F013273),2021-08-12,0.53,0.08,0.15,0.169880
4,010696,工银金融地产混合C(F010696),2020-11-24,1.25,-0.01,-0.01,0.190459


In [63]:
profit_df.to_excel('fund1.xlsx',encoding='utf8')

In [22]:
code_list[0]

'001594'

In [54]:
def automatic_investment_plan(code):
    '''
    定投收益
    '''
    df = get_net_value_df(code)

    money = 10000
    total_share =0 
    interval = 22
    length = len(df)
    sum_money=0
    count=0
    for i in range(0,length,interval):
        buy_date_df = df.iloc[i]
        share = money/buy_date_df['单位净值']
        total_share+=share
        sum_money+=money
        count+=1
    virtual_profit = (df.iloc[-1]['单位净值']*total_share-sum_money)/sum_money
    data=[]
    year,month,day=df.iloc[0]['净值日期'].split('-')
    for i in range(count):
        data.append((datetime.date(int(year), int(month), int(day))+datetime.timedelta(days=i*30), -1*money))
    current_money = df.iloc[-1]['单位净值']*total_share
    data.append((datetime.date(int(year), int(month), int(day))+datetime.timedelta(days=i*30),current_money))
    percent = xirr(data)
    every_round_profit = irr([money]*count+[-1*current_money])
    real_profit = pow(every_round_profit+1,count)-1
    return code,sum_money,round(current_money,2),round(percent,4),round(virtual_profit,4),round(real_profit,4)

In [55]:
def xirr(cashflows):
    # 函数
    years = [(ta[0] - cashflows[0][0]).days / 365. for ta in cashflows]
    residual = 1.0
    step = 0.05
    guess = 0.05
    epsilon = 0.0001
    limit = 10000
    while abs(residual) > epsilon and limit > 0:
        limit -= 1
        residual = 0.0
        for i, trans in enumerate(cashflows):
            residual += trans[1] / pow(guess, years[i])
        if abs(residual) > epsilon:
            if residual > 0:
                guess += step
            else:
                guess -= step
                step /= 2.0
    return guess - 1


In [ ]:
# 测试
data = [(datetime.date(2006, 1, 24), -39967), (datetime.date(2008, 2, 6), -19866), (datetime.date(2010, 10, 18), 245706), (datetime.date(2013, 9, 14), 52142)]
xirr(data)

In [56]:
result =[]
for code in code_list:
    d={}
    code,sum_money,current_money,percent,virtual_profit,real_profit=automatic_investment_plan(code)
    d['代码']=code
    d['投入总金额']=sum_money
    d['目前总金额']=current_money
    d['总收益']=percent
    d['简单收益率']=virtual_profit
    d['irr收益率']=real_profit
    result.append(d)
df = pd.DataFrame(result)


740000 891983.51
001594 0.06 0.21 0.43
740000 838011.42
502010 0.04 0.13 0.27
730000 548702.23
167301 -0.1 -0.25 -0.45
60000 62541.56
013273 0.22 0.04 0.07
140000 138765.67
010696 -0.02 -0.01 -0.02


In [53]:
df.head()

,代码,投入总金额,目前总金额,总收益,简单收益率,irr收益率
0,001594,740000,891983.51,0.06,20.54,0.43
1,502010,740000,838011.42,0.04,13.24,0.27
2,167301,730000,548702.23,-0.10,-24.84,-0.45
3,013273,60000,62541.56,0.22,4.24,0.07
4,010696,140000,138765.67,-0.02,-0.88,-0.02


In [57]:
df.to_excel('定投.xlsx',encoding='utf8')

In [43]:
def irr(values):
    """
    Return the Internal Rate of Return (IRR).
    .. deprecated:: 1.18
       `irr` is deprecated; for details, see NEP 32 [1]_.
       Use the corresponding function in the numpy-financial library,
       https://pypi.org/project/numpy-financial.
    This is the "average" periodically compounded rate of return
    that gives a net present value of 0.0; for a more complete explanation,
    see Notes below.
    :class:`decimal.Decimal` type is not supported.
    Parameters
    ----------
    values : array_like, shape(N,)
        Input cash flows per time period.  By convention, net "deposits"
        are negative and net "withdrawals" are positive.  Thus, for
        example, at least the first element of `values`, which represents
        the initial investment, will typically be negative.
    Returns
    -------
    out : float
        Internal Rate of Return for periodic input values.
    Notes
    -----
    The IRR is perhaps best understood through an example (illustrated
    using np.irr in the Examples section below).  Suppose one invests 100
    units and then makes the following withdrawals at regular (fixed)
    intervals: 39, 59, 55, 20.  Assuming the ending value is 0, one's 100
    unit investment yields 173 units; however, due to the combination of
    compounding and the periodic withdrawals, the "average" rate of return
    is neither simply 0.73/4 nor (1.73)^0.25-1.  Rather, it is the solution
    (for :math:`r`) of the equation:
    .. math:: -100 + \\frac{39}{1+r} + \\frac{59}{(1+r)^2}
     + \\frac{55}{(1+r)^3} + \\frac{20}{(1+r)^4} = 0
    In general, for `values` :math:`= [v_0, v_1, ... v_M]`,
    irr is the solution of the equation: [2]_
    .. math:: \\sum_{t=0}^M{\\frac{v_t}{(1+irr)^{t}}} = 0
    References
    ----------
    .. [1] NumPy Enhancement Proposal (NEP) 32,
       https://numpy.org/neps/nep-0032-remove-financial-functions.html
    .. [2] L. J. Gitman, "Principles of Managerial Finance, Brief," 3rd ed.,
       Addison-Wesley, 2003, pg. 348.
    Examples
    --------
    >>> round(np.irr([-100, 39, 59, 55, 20]), 5)
    0.28095
    >>> round(np.irr([-100, 0, 0, 74]), 5)
    -0.0955
    >>> round(np.irr([-100, 100, 0, -7]), 5)
    -0.0833
    >>> round(np.irr([-100, 100, 0, 7]), 5)
    0.06206
    >>> round(np.irr([-5, 10.5, 1, -8, 1]), 5)
    0.0886
    """
    # `np.roots` call is why this function does not support Decimal type.
    #
    # Ultimately Decimal support needs to be added to np.roots, which has
    # greater implications on the entire linear algebra module and how it does
    # eigenvalue computations.
    res = np.roots(values[::-1])  # 求根，对于n次多项式，p[0] * x**n + p[1] * x**(n-1) + ... + p[n-1]*x + p[n]，传入p的列表参数[p[0],p[1],...p[n]].
    mask = (res.imag == 0) & (res.real > 0)  # 虚部为0，实部为非负数。
    if not mask.any():  # 判断是否有满足条件的实根
        return np.nan  # 不满足，返回Not A Number
    res = res[mask].real
    # NPV(rate) = 0 can have more than one solution so we return
    # only the solution closest to zero.
    rate = 1/res - 1  # 这里解出的res，也就是符合条件的x，其实等于1/(1+r)，因此要做一个变换回去，r=1/x-1
    rate = rate.item(np.argmin(np.abs(rate)))  # argmin()取最小值的下标，也就是说可能会计算出多个折现率，我们取最小那个
    return rate

In [1]:
import requests

headers = {
    'authority': 'api.jiucaishuo.com',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json;charset=UTF-8',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
    'sec-ch-ua-platform': '"Linux"',
    'origin': 'https://www.funddb.cn',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'accept-language': 'zh,en;q=0.9,en-US;q=0.8,zh-CN;q=0.7',
}

data = '{"code":"013273","category":"wind_category","date":"","type":"pc","data_source":"xichou","version":"1.8.9","authtoken":"","act_time":1645634683356,"tirgkjfs":"fb","abiokytke":"52","u54rg5d":"76","kf54ge7":"f","tiklsktr4":"b","lksytkjh":"0ba1","sbnoywr":"27","bgd7h8tyu54":"46","y654b5fs3tr":"d","bioduytlw":"6","bd4uy742":"4","h67456y":"70b","bvytikwqjk":"46","ngd4uy551":"0b","bgiuytkw":"bc","nd354uy4752":"4","ghtoiutkmlg":"db8","bd24y6421f":"7e","tbvdiuytk":"7","ibvytiqjek":"1d","jnhf8u5231":"bc","fjlkatj":"760","hy5641d321t":"e4","iogojti":"e","ngd4yut78":"b8","nkjhrew":"4","yt447e13f":"1","n3bf4uj7y7":"b","nbf4uj7y432":"52","yi854tew":"54","h13ey474":"54f","quikgdky":"c3"}'

response = requests.post('https://api.jiucaishuo.com/v2/fund-lists/fundinvest', headers=headers, data=data)

In [4]:
result = response.json()

In [10]:
result['data']['gp']['list']

[{'name': '金地集团',
  'pe': '6.192465',
  'pb': '0.929627',
  'roe': '9.359700',
  'code': '600383',
  'xh': '1',
  'sw_category': '房地产',
  'zx_category': '房地产',
  'zz_category': '金融地产',
  'zjh_category': '房地产业',
  'wind_category': '房地产',
  'cc_count': 3,
  'desc': '',
  'sm': '增持',
  'incerease': '2.98%',
  'perect': '12.95%',
  'price': '12.98',
  'zdf': '-3.28',
  'pe_avg': 31.26,
  'pb_avg': 2.89,
  'roe_avg': '5.66%'},
 {'name': '华侨城A',
  'pe': '5.356094',
  'pb': '0.748560',
  'roe': '7.552000',
  'code': '000069',
  'xh': '2',
  'sw_category': '房地产',
  'zx_category': '房地产',
  'zz_category': '金融地产',
  'zjh_category': '房地产业',
  'wind_category': '可选消费',
  'cc_count': 13,
  'desc': '',
  'sm': '增持',
  'incerease': '3.86%',
  'perect': '12.50%',
  'price': '7.09',
  'zdf': '-3.67',
  'pe_avg': 31.26,
  'pb_avg': 2.89,
  'roe_avg': '5.66%'},
 {'name': '招商蛇口',
  'pe': '6.686332',
  'pb': '1.355274',
  'roe': '2.405300',
  'code': '001979',
  'xh': '3',
  'sw_category': '房地产',
  'zx_categ

In [6]:
def read_data_from_mongo(code):
    result=[]
    for item in fund[code].find():
        result.append(item)

    df = pd.DataFrame(result)
    return df

In [7]:
df = read_data_from_mongo(code_list[0])

In [8]:
df.head()

,_id,净值日期,单位净值,日增长率
0,621659cc28d482b10a0e5c05,2015-07-08,1.0000,0.0000
1,621659cc28d482b10a0e5c06,2015-07-10,1.0815,0.0000
2,621659cc28d482b10a0e5c07,2015-07-13,1.0514,-2.7832
3,621659cc28d482b10a0e5c08,2015-07-14,1.0324,-1.8071
4,621659cc28d482b10a0e5c09,2015-07-15,1.0498,1.6854


In [58]:
def get_max_withdraw(indexs):
    max_withdraw = 0
    start_date_index =0
    max_date_index =0
    last_high = indexs[0]
    
    for index,current in enumerate(indexs):
        # 遍历所有数据
        if current>last_high:
            last_high=current
            # start_date_index=index
            continue

        if (last_high-current)/last_high>max_withdraw:
            # 找到一个最大值时，保存其位置
            max_withdraw = (last_high-current)/last_high
            max_date_index=index

    return max_withdraw,max_date_index # 变成百分比